In [ ]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matlotlib inline
import seaborn as sns

from scipy import signal
from scipy import stats

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.statesspace.sarimax import SARIMAX



---
# User-defined Functions

In [ ]:
def fit_moving_average_trend(series, window=6):
#    return pd.rolling_mean(series, window, center=True)
    return series.rolling(window, center=True).mean()

def plot_moving_average_trend(ax, name, series, window=6):
    moving_average_trend = fit_moving_average_trend(series, window)
    plot_trend_data(ax, name, series)
    ax.plot(series.index.date, moving_average_trend)
    
def fit_seasonal_trend(series):
    dummies = create_monthly_dummies(series)
    X = sm.add_constant(dummies.values, prepend=False)
    seasonal_model = sm.OLS(series.values, X).fit()
    return seasonal_model.predict(X)

def plot_seasonal_trend(ax, name, series):
    seasons_average_trend = fit_seasonal_trend(series)
    plot_trend_data(ax, name, series, )
    ax.plot(series.index.date, seasons_average_trend, '-')

---
# Import Data

In [ ]:
caiso

### SET INDEX TO DATE

In [ ]:
np15_lmp = caiso['$_MWH_np15']
sp15_lmp = caiso['$_MWH_sp15']
zp26_lmp = caiso['$_MWH_zp26']

----
# Trend-Seasonal-Residual (TSR) Decomposition

In [ ]:
np15_decomp = sm.tsa.seasonal_decompose(np15_lmp)
sp15_decomp = sm.tsa.seasonal_decompose(sp15_lmp)
zp26_decomp = sm.tsa.seasonal_decompose(zp26_lmp)

In [ ]:
def plot_shared_yscales(axs, x, ys, titles):
    ymiddles =  [ (y.max()+y.min())/2 for y in ys ]
    yrange = max( (y.max()-y.min())/2 for y in ys )
    for ax, y, title, ymiddle in zip(axs, ys, titles, ymiddles):
        ax.plot(x, y)
        ax.set_title(title)
        ax.set_ylim((ymiddle-yrange, ymiddle+yrange))


def plot_seasonal_decomposition(axs, series, sd):
    plot_shared_yscales(axs,
                        series.index,
                        [series, sd.trend, sd.seasonal, sd.resid],
                        ["Raw Series", "Trend Component $T_t$", "Seasonal Component $S_t$", "Residual Component $R_t$"])

### NP15 TSR Decomposition

In [ ]:
fig, axs = plt.subplots(4, figsize(15,12), sharex=True)
plot_seasonal_decomposition(axs, np15_lmp, np15_decomp)
plt.tight_layout()

### SP15 TSR Decomposition

In [ ]:
fig, axs = plt.subplots(4, figsize(15,12), sharex=True)
plot_seasonal_decomposition(axs, sp15_lmp, sp15_decomp)
plt.tight_layout()

### ZP26 TSR Decomposition

In [ ]:
fig, axs = plt.subplots(4, figsize(15,12), sharex=True)
plot_seasonal_decomposition(axs, zp26_lmp, zp26_decomp)
plt.tight_layout()